# [Model Partner] Package a machine learning model for listing on Vulcan
* **conda_pytorch_p310**

The following diagram provides an overview of the ML model packaging process. In the diagram  step 1 you will store model artifacts and serving/scoring logic. In step 2 you create and push a container to ECR that is used to host your model on SageMaker which performs inference, and returns the prediction. In step 3 you validate the container can succesfully host your model on SageMaker. This notebook assumes step 1 to 3 are complete.


**Step 4** you will learn how to package the ML model into a Model Package. In **step 5** you will validate this ML model package by deploying it with Amazon SageMaker. In **step 6** you will learn about resources that guide you on how to list the ML model in AWS Marketplace.

<img src="images/ml-model-publishing-workflow.png"/>


**Table of contents**
1. [Step 4 - Create an ML Model Package](#step4):
    1. [Step 4.1 Define parameters](step41)
    1. [Step 4.1 Create Model Package](step42)
2. [Step 5 - Validate model in Amazon SageMaker environment](#step5): 
    1. [Step 5.1 Validate Real-time inference via Amazon SageMaker Endpoint](#step51)
7. [Step 6 - List ML model on AWS Marketplace](#step6)

## pre-request

**1. Remove/comment out the following code:**


```
380 if report.has_errors():
381                 raise ParamValidationError(report=report.generate_report())
```

Restart the Kernal, import boto3 again and re-run the cell. 


In [ ]:
import os

In [ ]:
strPythonPath = !which python
strValidatePath = os.path.join(strPythonPath[0].rsplit("/", 2)[0], "lib/python3.10/site-packages/botocore/validate.py")
print ("vi " + strValidatePath)

**2. Grant ECR permissiomn: AmazonEC2ContainerRegistryFullAccess**

In [ ]:
import sagemaker
print (f'role: {sagemaker.get_execution_role()}')

## AutoReload

In [ ]:
%load_ext autoreload
%autoreload 2

## 0. Install packages

In [ ]:
install_needed = True  # should only be True once
# install_needed = False

In [ ]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip uninstall pycodestyle -y

    IPython.Application.instance().kernel.do_shutdown(True)

## 1. Create an ML Model Artifact

### 1.1 model loading 

In [ ]:
strPrefix = "triton-ncf"
strModelName = "ncf_food_model"
strTrainedModelDir = "./custom-model"
strModelServingFolder = "triton-docker-serve-pt"

In [ ]:
from src.inference import model_fn

* 만약 prediction에 customization이 필요하다면, "./src/model.py"의 class NCF(nn.Module)의 forward 펑션 수정할 것 

In [ ]:
ncf_food_model = model_fn(strTrainedModelDir)

In [ ]:
ncf_food_model

### 1.2. Conversion to torchscript 

In [ ]:
import torch
import numpy as np

In [ ]:
def trace_model(mode, device, model, dummy_inputs, trace_model_name):

    model = model.eval()
    model.to(device)

    if mode == 'trace' : IR_model = torch.jit.trace(model, dummy_inputs)
    elif mode == 'script': IR_model = torch.jit.script(model)

    print(f"As {mode} : Model is saved {trace_model_name}")
    torch.jit.save(IR_model, trace_model_name)

    print("#### Load Test ####")    
    loaded_m = torch.jit.load(trace_model_name)    
    print(loaded_m.code)    
    dummy_user = dummy_inputs[0]
    dummy_item = dummy_inputs[1]    
    
    result = loaded_m(dummy_user, dummy_item)
    print("Result shape: ", result.shape) 

In [ ]:
is_trace, is_script = True, False

if is_trace: mode = 'trace'    
elif is_script: mode = 'script'

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

user_np = np.zeros((1,100)).astype(np.int32)
item_np = np.random.randint(low=1, high=1000, size=(1,100)).astype(np.int32)

dummy_inputs = [
    torch.from_numpy(user_np).to(device),
    torch.from_numpy(item_np).to(device)
]

In [ ]:
strTraceFoodModelName = 'ncf_food_model.pt'
trace_model(mode, device, ncf_food_model, dummy_inputs, strTraceFoodModelName) 

### 1.3.Create config.pbtxt

In [ ]:
%%writefile ncf_food_config.pbtxt

name: "ncf_food_model"
platform: "pytorch_libtorch"
max_batch_size: 128
input [
  {
    name: "INPUT__0"
    data_type: TYPE_INT32
    dims: [100]
  },
  {
    name: "INPUT__1"
    data_type: TYPE_INT32
    dims: [100]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [-1]
  }
]

### 1.4 Artifact packaging
- 아래와 닽은 폴더 구조를 생성해야 함.
```
model_serving_folder
    - model_name
        - version_number
            - model file
        - config file
        
# Example: 

triton-serve-pt
    - ncf_food
        - 1
            - model.pt
        - config.pbtxt

```

In [ ]:
import os
from utils.triton import copy_artifact

In [ ]:
# ncf_food_model 폴더 생성
food_config = 'ncf_food_config.pbtxt'
copy_artifact(strModelServingFolder, strModelName, strTraceFoodModelName, food_config)

### 1.5 Upload model packages

In [ ]:
import os
import sagemaker
from utils.triton import tar_artifact, upload_tar_s3

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
strModelTarFile = tar_artifact(strModelServingFolder, strModelName)    
print("strModelTarFile: ", strModelTarFile)
strModelUriPt = upload_tar_s3(sagemaker_session, strModelTarFile, strPrefix)
print("strModelUriPt: ", strModelUriPt)

### 1.6 Remove files

In [ ]:
listFilePath = [
    strTraceFoodModelName,
    f'{strModelName}.model.tar.gz',
    food_config
]
for strFilePath in listFilePath:
    if os.path.exists(strFilePath):
        os.remove(strFilePath)
    else:
        print("Can not delete the file as it doesn't exists")

## 2. Create custom docker image

In [ ]:
import boto3
from utils.ecr import ecr_handler
from utils.triton import account_id_map

In [ ]:
ecr = ecr_handler()
strAccountID = boto3.client("sts").get_caller_identity().get("Account")
strRegion = boto3.Session().region_name
strBucketName = sagemaker_session.default_bucket()
strExecutionRole = sagemaker.get_execution_role()

### 2.1 dockerfile

* Deep learning contatiners
    - https://github.com/aws/deep-learning-containers/blob/master/available_images.md
* Triton ver.
    - 23.01, 23.02, 23.03 and 22.07


In [ ]:
strBase = "amazonaws.com.cn" if strRegion.startswith("cn-") else "amazonaws.com"
strTritonImageUri = (
    "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.07-py3".format(
        account_id=account_id_map[strRegion],
        region=strRegion,
        base=strBase
    )
)
print(f'strtTritonImageUri: {strTritonImageUri}')

In [ ]:
%%writefile custom-docker/Dockerfile

FROM 785573368785.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:22.07-py3
RUN pip install -U pip
RUN pip install -U sagemaker
RUN pip install -U boto3
ENV PYTHONUNBUFFERED=TRUE

### 2.2 docker build

In [ ]:
strRepositoryName="js-onboarding"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerFile = "Dockerfile"
strDockerDir = "./custom-docker/"
strTag = "latest"

In [ ]:
ecr.build_docker(strDockerDir, strDockerFile, strRepositoryName, strRegionName="us-east-1", strAccountId="785573368785")

### 2.3 Push to ECR

In [ ]:
strEcrRepositoryUri = ecr.register_image_to_ecr(strRegion, strAccountID, strRepositoryName, strTag)

In [ ]:
#strEcrRepositoryUri = "419974056037.dkr.ecr.us-east-1.amazonaws.com/js-onboarding:latest"
print(f'strEcrRepositoryUri: {strEcrRepositoryUri}')

## 3. Validation (Serving and Inference)

In [ ]:
# Set to True to enable SageMaker to run locally
local_mode = True

if local_mode:
    
    from sagemaker.local import LocalSession
    
    strInstanceType = "local_gpu"
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    
else:
    strInstanceType = "ml.m5.2xlarge" #"ml.p3.2xlarge"#"ml.g4dn.8xlarge"#"ml.p3.2xlarge", 'ml.p3.16xlarge' , ml.g4dn.8xlarge
    sagemaker_session = sagemaker.Session()

nInstanceCount = 1

### 3.1 Local mode
- 내부적으로 Triton 서버가 구동시에 아래 URL 스크립트가 구동 됨.
    - 여기에 맞는 필요한 환경 변수를 넣어 줌.
    - https://raw.githubusercontent.com/triton-inference-server/server/main/docker/sagemaker/serve

#### 3.1.1 Depoly

In [ ]:
import time
import json
import numpy as np
from sagemaker.model import Model

In [ ]:
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# endpoint variables
strSMModelName = f"{strPrefix}-mdl-{ts}" #sm_model_name
strEndpointConfigName = f"{strPrefix}-epc-{ts}" # endpoint_config_name
strEndpointName = f"{strPrefix}-ep-{ts}" # endpoint_name
strModelDataUrl = f"s3://{strBucketName}/{strPrefix}/" #model_data_url

In [ ]:
dicContainerEnvs = {
                    "SAGEMAKER_TRITON_LOG_VERBOSE": "3",
                    "SAGEMAKER_TRITON_LOG_INFO": "1",
                    "SAGEMAKER_TRITON_LOG_WARNING" : "1",
                    "SAGEMAKER_TRITON_LOG_ERROR" : "1"
                 }

localPytorchModel = Model(
    model_data= strModelUriPt,
    image_uri = strEcrRepositoryUri,
    role=strExecutionRole,
    env = dicContainerEnvs
)

In [ ]:
localPredictor = localPytorchModel.deploy(
    instance_type=strInstanceType,
    initial_instance_count=1,
    endpoint_name=strEndpointName,
    wait=True,
    log=False,
)

#### 3.1.2 Inference

In [ ]:
def create_sample_payload():
    # user
    user_np = np.zeros((1,100)).astype(np.int32)
    # item
    item_np = np.random.randint(low=1, high=1000, size=(1,100)).astype(np.int32)

    payload = {
        "inputs": [
            {"name": "INPUT__0", "shape": [1,100], 
             "datatype": "INT32", "data": user_np.tolist()},
            {"name": "INPUT__1", "shape": [1,100], 
             "datatype": "INT32", "data": item_np.tolist()},
        ]
    }
    
    return payload

payload = create_sample_payload()
print("payload: ", payload)

In [ ]:
def single_model_invoke_endpoint(client,endpoint_name, payload): 
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/octet-stream", 
        Body=json.dumps(payload),
    )

    result = json.loads(response["Body"].read().decode("utf8"))
    
    return result

runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
result = single_model_invoke_endpoint(runtime_client,strEndpointName, payload)
print("result : ", result)

#### 3.1.3 Delete endpoint

In [ ]:
from utils.inference_utils import delete_endpoint

In [ ]:
client = sagemaker.local.LocalSagemakerClient()
delete_endpoint(client, strEndpointName)

### 3.2 Cloud mode

**local_mode = True 변경하기**

#### 3.2.1 Depoly

In [ ]:
dicContainer = {
    "Image": strEcrRepositoryUri,
    "ModelDataUrl": strModelUriPt
}

In [ ]:
print(f'dicContainer: {dicContainer}')
print(f'strSMModelName: {strSMModelName}')

In [ ]:
sm_client = boto3.client(service_name="sagemaker")

create_model_response = sm_client.create_model(
    ModelName=strSMModelName,
    ExecutionRoleArn=strExecutionRole,
    PrimaryContainer=dicContainer
)

In [ ]:
print(f'Model Arn: {create_model_response["ModelArn"]}')

In [ ]:
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=strEndpointConfigName,
    ProductionVariants=[
        {
            "InstanceType": strInstanceType,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": strSMModelName,
            "VariantName": "AllTraffic",
        }
    ],
)

In [ ]:
print(f'Endpoint Config Arn: {create_endpoint_config_response["EndpointConfigArn"]}')

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=strEndpointName,
    EndpointConfigName=strEndpointConfigName
)

In [ ]:
print(f'Endpoint Arn: {create_endpoint_response["EndpointArn"]}')

In [ ]:
%%time 

resp = sm_client.describe_endpoint(EndpointName=strEndpointName)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=strEndpointName)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

#### 3.2.2 Inference

In [ ]:
runtime_client = boto3.Session().client('sagemaker-runtime')
single_model_invoke_endpoint(runtime_client,strEndpointName, payload)

#### 3.2.3 Delete endpoint

In [ ]:
client = boto3.Session().client('sagemaker')
delete_endpoint(client, strEndpointName)

## 4. Create an ML Model Package

In [ ]:
import json
import boto3
import sagemaker as sage
from sagemaker import get_execution_role, ModelPackage
import time

# Common variables
session = sage.Session()
s3_bucket = session.default_bucket()
region = session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

s3_client = session.boto_session.client("s3")
sm_runtime = boto3.client("sagemaker-runtime")

In this section, we will see how you can package your artifacts (ECR image and the trained model artifacts) into a ModelPackage. Once you complete this, you can list your product as a pretrained model in the AWS Marketplace.

**NOTE:** If your model can be deployed on multiple hardware types (CPU/GPU/Inferentia) then a ModelPackage must be created for each and added to the MP listing as different versions as, in general, the container image used will be different for each.  

#### Model Package Definition
A Model Package is a reusable abstraction for model artifacts that packages all the ingredients necessary for inference. It consists of an inference specification that defines the inference image to use along with an optional model data location.

The ModelPackage must be created in the AWS account that will be registered as a seller on the AWS Marketplace.

### 4.1 Define parameters 

In [ ]:
# Define parameters
model_name = "marketplace-model-test"#"<<YourModelName>>"
model_description = "marketplace-model-test"#"<<YourModelDescription>>"

# <<YourSupportedContentTypes>>
supported_content_types = ["application/octet-stream"]#["text/csv", "application/json", "application/jsonlines"]

# <<YourSupportedResponseMIMETypes>>
supported_response_MIME_types = [ 
    "application/json"
]

A Model Package creation process requires you to specify following:
  1. Docker image
  2. Model artifacts
    - You can either package these inside the docker image, as we have done in this example, or provide them as a gzipped tarball.
    - In the case of large Models gzipped tarball is required. 
  3. Validation specification 
        
In order to provide confidence to sellers (and buyers) that the products work in Amazon SageMaker, before listing them on AWS Marketplace SageMaker needs to perform basic validations. The product can be listed in AWS Marketplace only if this validation process succeeds. This validation process uses the validation profile and sample data provided by you to create a transform job in your account using the Model to verify your inference image works with SageMaker.

Next, you need to identify the right instance-sizes for your ML models. You can do so by running performance tests on top of your ML Model.

**NOTE:** In addition to tuning, take into account the requirements of your model when identifying instance types.  If your model does not use GPU resources, then do not include GPU instance types.  Similarly, if your model does use GPU resources, but can only make use of a single GPU, do not include instance types that have multiple GPUs as it will lead to increased infrastructure charges for your customers with no performance benefit.

In [ ]:
supported_realtime_inference_instance_types = ["ml.g5.12xlarge"]#["<<YourModelSupportInstanceType>>"]
supported_batch_transform_instance_types = ["ml.m5.xlarge"] #  use either a g4dn.12xlarge or p3.8xlarge. However, the Batch Transform validation step is not required

In [ ]:
validation_file_name = "input.jsonl"
validation_input_path = f"s3://{s3_bucket}/validation-input-json/"
validation_output_path = f"s3://{s3_bucket}/validation-output-jsonl/"

*(Not required)* First, we create sample data to be used in the validation stage of the ModelPackage creation and upload it to S3. This sample data would need to be in the format your model expects

In [ ]:
json_line = json.dumps(payload)
with open("input.jsonl", "w") as f:
    f.write(json_line)
s3_client.put_object(Bucket=s3_bucket, Key="validation-input-json/input.jsonl", Body=json_line)

In [ ]:
print (strEcrRepositoryUri)
print (strModelUriPt)

### 4.2 Create Model Package 

In [ ]:
#docker_image_uri = "419974056037.dkr.ecr.us-east-1.amazonaws.com/js-onboarding:latest"
#model_data_location = "s3://sagemaker-us-east-1-419974056037/triton-ncf/ncf_food_model.model.tar.gz"

In [ ]:
docker_image_uri = strEcrRepositoryUri#"<<YourModelImageURI>>" # ECR URI of Image used to host model
model_data_location = strModelUriPt#"<<YourModelS3Location>>"

When creating the ModelPackage you will recieve the error:

```
~/anaconda3/envs/python3/lib/python3.8/site-packages/botocore/validate.py in serialize_to_request(self, parameters, operation_model)
    380             if report.has_errors():
--> 381                raise ParamValidationError(report=report.generate_report())
    382         return self._serializer.serialize_to_request(
    383             parameters, operation_model

ParamValidationError: Parameter validation failed:
Invalid length for parameter ValidationSpecification.ValidationProfiles, value: 0, valid min length: 1
```

In order to resolve this issue, open the `validate.py` file in this case it is located at `~/anaconda3/envs/python3/lib/python3.8/site-packages/botocore/validate.py`. Remove/comment out the following code: 

```
380 if report.has_errors():
381                 raise ParamValidationError(report=report.generate_report())
```

Restart the Kernal, import boto3 again and re-run the cell. 



**!which python** <BR>
**for pytorch_p310: cd ~/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/validate.py**

In [ ]:
model_package = session.sagemaker_client.create_model_package(
    ModelPackageName=model_name,
    ModelPackageDescription=model_description,
    InferenceSpecification={
        "Containers": [
            {
                "Image": docker_image_uri,
                "ModelDataUrl": model_data_location
            }
        ],
        "SupportedTransformInstanceTypes": supported_batch_transform_instance_types,
        "SupportedRealtimeInferenceInstanceTypes": supported_realtime_inference_instance_types,
        "SupportedContentTypes": supported_content_types,
        "SupportedResponseMIMETypes": supported_response_MIME_types,
    },
    CertifyForMarketplace=True,  # Make sure to set this to True
    #ValidationSpecification={
    #    "ValidationRole": role,
    #    "ValidationProfiles": [],
    #},
    ValidationSpecification={
        'ValidationRole': role,
        'ValidationProfiles': [
            {
                'ProfileName': "validation",
                'TransformJobDefinition': {
                    'MaxConcurrentTransforms': 1,
                    'MaxPayloadInMB': 64,
                    'BatchStrategy': 'SingleRecord',
                    #'Environment': {
                    #    'string': 'string'
                    #},
                    'TransformInput': {
                        'DataSource': {
                            'S3DataSource': {
                                'S3DataType': 'S3Prefix',
                                'S3Uri': f'{validation_input_path}input.jsonl'
                            }
                        },
                        'ContentType': 'application/octet-stream',
                        'CompressionType': 'None',
                        'SplitType': 'None'
                    },
                    'TransformOutput': {
                        'S3OutputPath': f'{validation_output_path}output.json',
                        'Accept': 'application/json',
                        'AssembleWith': 'None',
                        #'KmsKeyId': 'string'
                    },
                    'TransformResources': {
                        'InstanceType': 'ml.m5.xlarge',
                        'InstanceCount': 1,
                        #'VolumeKmsKeyId': 'string'
                    }
                }
            },
        ]
    },
)

In [ ]:
session.wait_for_model_package(model_package_name=model_name) # If failure occurs navigate to SageMaker Console > My marketplace model packages > select the failed ModelPackage for details. 

Once you have executed the preceding cell, open the [Model Packages console from Amazon SageMaker](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources) and check if model creation succeeded. 

Choose the Model and then open the **Validation** tab to see the validation results.

## 5. Validate model in Amazon SageMaker environment

##### Create a deployable model from the model package.

In [ ]:
model = ModelPackage(
    role=role,
    model_package_arn=model_package["ModelPackageArn"],
    sagemaker_session=session,
)

### 5.1 Validate Real-time inference via Amazon SageMaker Endpoint

##### Deploy the SageMaker model to an endpoint

In [ ]:
model.deploy(
    initial_instance_count=1,
    instance_type=supported_realtime_inference_instance_types[0],
    endpoint_name=model_name,
)
model.endpoint_name

In [ ]:
content_type = supported_content_types[0]

##### Example invocation via boto3

In [ ]:
# Make use of your own example input data to test the Endpoint
#input_json = '{"text": "sample"}'

response = sm_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType=content_type,
    Accept="application/json",
    Body=json.dumps(payload),
)

json.load(response["Body"])

##### Example invocation via the AWS CLI

In [ ]:
# Perform inference
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model.endpoint_name \
    --body fileb://$validation_file_name \
    --content-type $content_type \
    --region $session.boto_region_name \
    out.out
    
    
# Print inference
!head out.out

Clean up the endpoint and endpoint configuration created.

In [ ]:
model.sagemaker_session.delete_endpoint(model.endpoint_name)
model.sagemaker_session.delete_endpoint_config(model.endpoint_name)

Congratulations! Since the model is not required, you can delete it. Note that you are deleting the deployable model. Not the model package.

In [ ]:
model.delete_model()

To publish the model to the AWS Marketplace, you will need to specify model package ARN. Copy the following Model Package ARN 

In [ ]:
model_package["ModelPackageArn"]

### <a name="step6"></a>Step 6: List ML Model on AWS Marketplace


1. Model Partner creates [public profile](https://docs.aws.amazon.com/marketplace/latest/userguide/seller-registration-process.html#seller-public-profile) on AWS Marketplace and registers to be a seller.
There is no need to provide Tax information as the product on Marketplace will be listed as free.

2. In the [Model Packages](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources) section of the SageMaker console you'll find the entity you created in this notebook. If it was successfully created and validated, you should be able to select the entity and choose **Publish new ML Marketplace listing**.

<img src="images/publish-to-marketplace-action.png"/>

You will be redirected to the [AWS Marketplace Management portal](https://aws.amazon.com/marketplace/management/ml-products/) where you will be able to build a listing.

<img src="images/listing.png"/>

1. If your model targets multiple hardware types, remember to add each ModelPackage to the listing as separate versions.
2. Click Add and fill in the model information. Kindly set Product visibility must be set to `Public`.
<img src="images/public.png"/>

3. Allowlist account `171503325295`, `572320329544` and `559110549532` for access to the model. 
For region support select: `us-east-1, us-west-2, eu-west-1, eu-central-1, eu-west-2, ap-northeast-1, ap-south-1, ca-central-1, us-east-2, ap-northeast-2`
<img src="images/allowlist-accs.png"/>

4. Under Pricing and terms, set pricing model as:
**Inference based pricing (custom metering) at $0**

You will see the following:
(Optional) If the container did not implement the below please confirm and move forward. 
```
I confirm that my model package supports the response header for custom metering. Example response header: X-Amzn-Inference-Metering:
{"Dimension": "inference.count", "ConsumedUnits": 3}
I understand that in absence of this header, default metering will be used instead.
```

<img src="images/inference-based-pricing.png"/>

5. Listing status should show as follows:
**Do not click Sign off and publish**

<img src="images/status-1.png"/>

6. Vissibility status of the listing should be `Limited`.

<img src="images/status-2.png"/>




**Resources**
* [Publishing your product in AWS Marketplace](https://docs.aws.amazon.com/marketplace/latest/userguide/ml-publishing-your-product-in-aws-marketplace.html)


In [ ]:
!which python